In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import wntr
from pathlib import Path
import wntr.network.controls as controls
from dreaminsg_integrated_model.network_sim_models.interdependencies import *

In [7]:
network_dir= 'in2'

MAIN_DIR = Path('..')
#water_file = MAIN_DIR/'dreaminsg_integrated_model/data/networks/examples/water/networks/Net1.inp'
water_file = MAIN_DIR/'dreaminsg_integrated_model/data/networks/{}/water/Example_water2.inp'.format(network_dir)

step_size = 120
try:
    wn = wntr.network.WaterNetworkModel(water_file)
    wn.options.time.duration = step_size
    wn.options.time.report_timestep = step_size
    wn.options.time.hydraulic_timestep = step_size
    wn.options.hydraulic.demand_model = 'PDA'
    total_base_water_demand = sum([wn.get_node(node).base_demand for node in wn.junction_name_list])
    print("Water network successfully loaded.")
except FileNotFoundError:
    print("Error: The water network file does not exist. No such file or directory: ", water_file)
    
time_list = [step_size*70*i for i in range(0,10)]
W_WP9_perf = [100, 100, 100, 25, 25, 100, 100]

for i in range(0,5):
    print(i)
    if W_WP9_perf[i] == 100:
        wn.get_link('W_WP9').status = 1
#     else:
        wn.get_link('W_WP9').status = 0
    print("Simulation time: ", wn.options.time.duration,"; Hydraulic time step: ", wn.options.time.hydraulic_timestep, "; Report time step: ", wn.options.time.report_timestep)
    wn_sim = wntr.sim.WNTRSimulator(wn)
    wn_results = wn_sim.run_sim(convergence_error=True, solver_options = {'MAXITER': 10000})

    wn.options.time.duration += time_list[i]
    wn.options.time.report_timestep += time_list[i]
    
    print("Performance of '9' is", W_WP9_perf[i])
    print("Pump: ", "\t\tstatus = ", wn_results.link['status']['W_WP9'].values, "\tflowrate = ", wn_results.link['flowrate']['W_WP9'].values)
    print("Tank: ", "\t\tdemand", wn_results.node['demand']['W_T2'].values, "\thead = ", wn_results.node['head']['W_T2'].values)
    print("Pipe from Tank: ", "status", wn_results.link['status']['W_P110'].values, "\tflowrate = ", wn_results.link['flowrate']['W_P110'].values)
    print("******************\n")


Water network successfully loaded.
0
Simulation time:  120.0 ; Hydraulic time step:  120.0 ; Report time step:  120
Performance of '9' is 100
Pump:  		status =  [0 0] 	flowrate =  [0. 0.]
Tank:  		demand [-0.069 -0.069] 	head =  [295.656 295.611]
Pipe from Tank:  status [1 1] 	flowrate =  [0.069 0.069]
******************

1
Simulation time:  120.0 ; Hydraulic time step:  120.0 ; Report time step:  120
Performance of '9' is 100
Pump:  		status =  [0] 	flowrate =  [0.]
Tank:  		demand [-0.069] 	head =  [295.566]
Pipe from Tank:  status [1] 	flowrate =  [0.069]
******************

2
Simulation time:  8520.0 ; Hydraulic time step:  120.0 ; Report time step:  8520
Performance of '9' is 100
Pump:  		status =  [1] 	flowrate =  [0.15]
Tank:  		demand [0.08] 	head =  [292.758]
Pipe from Tank:  status [1] 	flowrate =  [-0.08]
******************

3
Simulation time:  25320.0 ; Hydraulic time step:  120.0 ; Report time step:  25320
Performance of '9' is 25
Pump:  		status =  [1] 	flowrate =  [0.14]

In [14]:
network_dir= 'in2'

MAIN_DIR = Path('..')
water_file = MAIN_DIR/'dreaminsg_integrated_model/data/networks/examples/water/networks/Net1.inp'
#water_file = MAIN_DIR/'dreaminsg_integrated_model/data/networks/{}/water/Example_water2.inp'.format(network_dir)

step_size = 120
try:
    wn = wntr.network.WaterNetworkModel(water_file)
    wn.options.time.duration = step_size
    wn.options.time.report_timestep = step_size
    wn.options.time.hydraulic_timestep = step_size
    wn.options.hydraulic.demand_model = 'PDA'
    total_base_water_demand = sum([wn.get_node(node).base_demand for node in wn.junction_name_list])
    print("Water network successfully loaded.")
except FileNotFoundError:
    print("Error: The water network file does not exist. No such file or directory: ", water_file)

interval_list = [step_size*70*i for i in range(1,10)]


W_WP9_perf = [100, 25, 25, 25, 25, 25, 100, 100]

# pump = wn.get_link('9')
# pump.add_outage(wn, 120 + sum(interval_list[0:1]), 120 + sum(interval_list[0:6]))

# pump = wn.get_link('9')
# tankfull_act = controls.ControlAction(pump, 'status', 0)
# tank = wn.get_node('2')
# tankfull_cond = controls.ValueCondition(tank, 'level', '<',tank.min_level)
# tankfull_ctrl = controls.Control(tankfull_cond, tankfull_act, name='tankfull_ctrl')

total_demand = []

for i in range(0,8):
    print(i)
    if W_WP9_perf[i] == 100:
        pass
    else:
        pump = wn.get_link('9')
        pump.add_outage(wn, sum(interval_list[0:i]), sum(interval_list[0:i+1]))
    print("Simulation time: ", wn.options.time.duration,"; Hydraulic time step: ", wn.options.time.hydraulic_timestep, "; Report time step: ", wn.options.time.report_timestep)
    wn_sim = wntr.sim.WNTRSimulator(wn)
    wn_results = wn_sim.run_sim(convergence_error=True, solver_options = {'MAXITER': 5000})

    wn.options.time.duration += interval_list[i]
    #wn.options.time.hydraulic_timestep += 60.0
    wn.options.time.report_timestep += interval_list[i]
    print("Performance of '9' is", W_WP9_perf[i])
    print(wn_results.link['status'])
    
    
    
    print("Pump: ", "\t\tstatus = ", wn_results.link['status']['9'].values, "\tflowrate = ", wn_results.link['flowrate']['9'].values)
    print("Tank: ", "\t\tdemand", wn_results.node['demand']['2'].values, "\thead = ", wn_results.node['head']['2'].values)
    print("Pipe from Tank: ", "status", wn_results.link['status']['110'].values, "\tflowrate = ", wn_results.link['flowrate']['110'].values)
    print("******************\n")

Water network successfully loaded.
0
Simulation time:  120.0 ; Hydraulic time step:  120.0 ; Report time step:  120
Performance of '9' is 100
     10  11  12  21  22  31  110  111  112  113  121  122  9
0     1   1   1   1   1   1    1    1    1    1    1    1  1
120   1   1   1   1   1   1    1    1    1    1    1    1  1
Pump:  		status =  [1 1] 	flowrate =  [0.118 0.118]
Tank:  		demand [0.048 0.048] 	head =  [295.656 295.687]
Pipe from Tank:  status [1 1] 	flowrate =  [-0.048 -0.048]
******************

1
Simulation time:  8520.0 ; Hydraulic time step:  120.0 ; Report time step:  8520
Performance of '9' is 25
      10  11  12  21  22  31  110  111  112  113  121  122  9
8520   1   1   1   1   1   1    1    1    1    1    1    1  0
Pump:  		status =  [0] 	flowrate =  [0.]
Tank:  		demand [-0.083] 	head =  [297.641]
Pipe from Tank:  status [1] 	flowrate =  [0.083]
******************

2
Simulation time:  25320.0 ; Hydraulic time step:  120.0 ; Report time step:  25320
Performance of '

<Pump '9' from '9' to '10', pump_type='HEAD', pump_curve=1, speed=<TimeSeries: base=1.0, pattern=None, category='None'>, status=Opened>

In [11]:
wn.get_node('2').perf = 100
wn.get_node('2').perf

100

In [12]:
pn.motor

NameError: name 'pn' is not defined